# 1. Introduction

## 1.1 Imports
Import libraries here.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from kuberspatiotemporal import CompoundModel, Feature, SpatialModel, KuberModel
from kuberspatiotemporal.tools import make_ellipses

In [ ]:
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import FunctionTransformer

# 2. Load Data

In [ ]:
data = pd.read_json('data/adriana_dev.json')

In [ ]:
data.head(2)

In [ ]:
#data = data[data['timestamp']>'2020-03-14 09:33:14+00:00']

In [ ]:
from ipyleaflet import Map, basemaps, basemap_to_tiles, Heatmap

m = Map(center=(np.mean(data.latitude), np.mean(data.longitude)), zoom=10)
heatmap = Heatmap(
    locations=data[['latitude', 'longitude']].values.tolist(),
    radius=20
)
m.add_layer(heatmap)

m

# 3. Learn Spatiotemporal Model - 4D

In [ ]:
data['time'] = [ts.hour + ts.minute/60 + ts.second/3600 for ts in data.timestamp]
data['weekday'] = [ts.dayofweek for ts in data.timestamp]

In [ ]:
data

In [ ]:
limits = [np.min(data[['latitude', 'longitude', 'time']].values, axis=0),np.max(data[['latitude', 'longitude', 'time']].values, axis=0)]

In [ ]:
limits

In [ ]:
kst = CompoundModel(
    n_dim=4,
    n_iterations=100,
    scaling_parameter=1.1,
    nonparametric=True,
    online_learning=False,
    loa=True,
    features=[
        Feature(SpatialModel(n_dim=3, min_eigval=1e-9, limits=limits), [0, 1, 2]),
        Feature(KuberModel(n_symbols=7), [3])
    ],
)

In [ ]:
pipeline = make_pipeline(
    make_column_transformer(
        (FunctionTransformer(lambda x: np.array(x).reshape(-1, 1)), "latitude"),
        (FunctionTransformer(lambda x: np.array(x).reshape(-1, 1)), "longitude"),
        (FunctionTransformer(lambda x: np.array(x).reshape(-1, 1)), "time"),
        (FunctionTransformer(lambda x: np.array(x).reshape(-1, 1)), "weekday"),
    ),
    kst,
)

In [ ]:
pipeline.fit(data[['latitude', 'longitude', 'time', 'weekday']])

In [ ]:
from typing import (
    Iterable,
    Tuple,
    Union,
    NamedTuple,
    List
)

import numpy as np


def subdivide(lower: np.ndarray, upper: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Uniformely subdivides an interval into four smaller intervals
    [extended_summary]
    Parameters
    ----------
    lower : np.ndarray
        (2,) lower bounds of the interval
    upper : np.ndarray
        (2,) upper bounds of the interval
    Returns
    -------
    Tuple[np.ndarray,np.ndarray]
        Returns new the new lower and upper bounds (4,2) as well as the new widths (2,)
    """

    width = (upper - lower)/2
    lowers = np.array(
        [
            lower,
            np.array([lower[0], lower[1] + width[1]]),
            np.array([lower[0] + width[0], lower[1]]),
            lower + width,
        ]
    )
    uppers = lowers + width

    return lowers, uppers, width

class Leaf(NamedTuple):
    """Class representing a leaf in a quadtree."""
    lower: np.ndarray
    upper: np.ndarray
    probability: float


class Quadtree:
    """Class that represents a quadtree."""

    def __init__(self,
                 model,
                 t: float,
                 time,
                 day):
        self.model = model
        self.min_prob = t
        self.leaves = []
        self.threshold = t
        self.time = time
        self.day = day

    def start(self,
              lower: np.ndarray,
              upper: np.ndarray):
        """
        Add a top level leaf to the tree

        When you want to continue computation, just call descent multiple times.
        When loading from disc, write the leaves manually in the `self.leaves` attribute.

        Parameters
        ----------
        lower : np.ndarray
            lower interval boundary
        uppe : np.ndarray
            upper interval boundary
        """

        # why 1.0? laziness here but we can survive to make the very first subdivision in any case
        self.leaves.append(Leaf(lower, upper, 1.0))

    def expand(self, leaf: Leaf) -> List[Leaf]:
        """
        Expands a leaf and returns a list of the children or
        a list with itself (if the probability is below the threshold)

        Parameters
        ----------
        leaf : Leaf
            The leaf to expand

        Returns
        -------
        List[Leaf]
            Either list of four children or the initial leaf itself
        """
        if leaf.probability < self.threshold:
            return [leaf]
        else:
            lowers, uppers, delta = subdivide(leaf.lower, leaf.upper)
            return [Leaf(lower, upper, self.model.score(np.concatenate((lower+delta/2, np.array([self.time,self.day])))[np.newaxis,:])) for lower, upper in zip(lowers, uppers)]


    def descent(self, iterations: int):
        """
        (Continue to) iterate over the leaves

        [extended_summary]

        Parameters
        ----------
        iterations : int
            Number of iterations steps (refers to tree depth)
        """

        for i in range(iterations): # pylint: disable=unused-variable

            # expand all leaves returns a list of lists
            expanded = [self.expand(leaf) for leaf in self.leaves]

            # Recipe of how to flatten a list of lists:
            # flat_list = [item for sublist in l for item in sublist]
            self.leaves = [item for sublist in expanded for item in sublist]


In [ ]:
from ipyleaflet import Map, CircleMarker, Rectangle, LayerGroup
from matplotlib.colors import to_hex
from ipywidgets import interact, interactive, interact_manual, HTML
import matplotlib.pyplot as plt
import time

def cndtn(day=2, hour=10, i=5):
    """Helper plot function for displaying a conditional model"""
    start = time.time()
    world.remove_layer(layer_rectangles)

    layer_samples.clear_layers()
    layer_rectangles.clear_layers()


    # Acquire some sample for getting the boundaries
    samples = kst.rvs(10000)
    samples = samples[(samples[:,3]==day) & (samples[:,2]>=hour) & (samples[:,2]<hour+1)][:,0:2]

    # pick min and max values but widen interval by 25%
    max_ = np.max(samples, axis=0)
    min_ = np.min(samples, axis=0)

    width = max_ - min_

    max_ += width * 0.25
    min_ -= width * 0.25
    width = max_ - min_

    cmap = plt.cm.get_cmap("viridis")
    start1 = time.time()
    
    # Create Quadtree Object
    kst.loa = False
    kst.score_threshold = None
    kst.quantiles = kst.get_score_threshold(data[['latitude', 'longitude', 'time', 'weekday']].values, lower_quantile=0, upper_quantile=0.3)

    tree = Quadtree(model = kst, t = 0.005, time=hour, day=day)
    tree.start(min_, max_)
    tree.descent(i)
    prob = 0
    for leaf in tree.leaves:
        
        message = HTML()
        message.value = f"{leaf.probability}"
        prob += leaf.probability
        color = cmap(leaf.probability)

        color_hex = to_hex(color, keep_alpha=False)

        rectangle = Rectangle(
            bounds=(leaf.lower.tolist(), (leaf.upper.tolist())),
            weight=1,
            fill_color=color_hex,
            stroke=True,
            fill_opacity=0.5,
        )

        layer_rectangles.add_layer(rectangle)
        rectangle.popup = message

    world.add_layer(layer_rectangles)
    print("Total probability: %s" % (prob))
    print("Granularity (max number of squares): %s" % (4 ** i))
    print("Grid Computation time: %s sec" % (time.time() - start1))
    print("Viz+Computation time: %s sec" % (time.time() - start))

In [ ]:
center = np.mean(data[['latitude', 'longitude']], axis=0).tolist()
world = Map(center=center, zoom=13)
world.layout.width = "1200px"
world.layout.height = "600px"
layer_samples = LayerGroup()
layer_rectangles = LayerGroup()

world.add_layer(layer_rectangles)
world

In [ ]:
interactive_plot = interactive(cndtn, hour=(0, 24), day=(0, 6, 1), i=(0, 20))
display(interactive_plot)